# Rebuild a noise-free covariance matrix to include noise

In [16]:
import crosspower as cp
import FisherCl as fcl # branch quickCl
import noiseCl as ncl
#import camb
import os

# set plotting to be in the notebook instead of its own qt window
%matplotlib inline

# Load a Noise-Free k,g + T,E Fisher Matrix

In [17]:
# load saved Fobj
# get lensing and galaxy Fisher matrix to add

import FisherCl as fcl
import noiseCl as ncl
import crosspower as cp
import pickle

#filehandler = open('Fisher_7_6X4_w2_wa.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
#filehandler = open('Fisher_7_6X4_w2.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
#filehandler = open('Fisher_7_6X4.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
#neutrino_hierarchy = 'normal', usePrimaryCMB = True; from FisherCl branch quickCl
# kludge for this one: told cp.getDNDZinterp to use modelDNDZ3; set cp.tophat beesBins=True
# biasByBin = False (uses sigma(b)/b), dw = 0.005

filehandler = open('Fisher_7_6X2_w2_wa.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=True,
#filehandler = open('Fisher_7_6X2_w2.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=True,
#filehandler = open('Fisher_7_6X2.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=True,
#neutrino_hierarchy = 'normal', usePrimaryCMB = True; from FisherCl branch quickCl
# kludge for this one: told cp.getDNDZinterp to use modelDNDZ3; set cp.tophat beesBins=True
# biasByBin = False (uses sigma(b)/b), dw = 0.005

# this one has TE polarazation to l=5000, but only one galaxy bin
#filehandler = open('Fisher_4_1Y.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000; nz=3e5; nonlinear=True,
#neutrino_hierarchy = 'normal', usePrimaryCMB = True; from FisherCl branch quickCl
# this one used fiducial params and values to match CMB-S4 Science book verification table 8.1

Fobj = pickle.load(filehandler) 


lmin = Fobj.lmin
lmax = Fobj.lmax
nMaps = Fobj.nBins + 1
nParams = Fobj.nParams
nCosParams = Fobj.nParams - Fobj.nBins
ells = Fobj.ells
obsNames = Fobj.obsNames
paramList = Fobj.paramList

print Fobj.obsList
print Fobj.obsNames
print Fobj.paramList
print Fobj.obsListP
print Fobj.obsNamesP

print Fobj.Fij.shape
print Fobj.FijTE.shape
print Fobj.nParams
print Fobj.nCosParams

#FijTE0 = Fobj.FijTE
#invCovP = Fobj.invCovP

[ 0.  1.  2.  3.  4.  5.  6.  8.  9. 10. 11. 12. 13. 16. 17. 18. 19. 20.
 24. 25. 26. 27. 32. 33. 34. 40. 41. 48.]
['k,k', 'k,g1', 'k,g2', 'k,g3', 'k,g4', 'k,g5', 'k,g6', 'g1,g1', 'g1,g2', 'g1,g3', 'g1,g4', 'g1,g5', 'g1,g6', 'g2,g2', 'g2,g3', 'g2,g4', 'g2,g5', 'g2,g6', 'g3,g3', 'g3,g4', 'g3,g5', 'g3,g6', 'g4,g4', 'g4,g5', 'g4,g6', 'g5,g5', 'g5,g6', 'g6,g6']
['ombh2', 'omch2', 'cosmomc_theta', 'As', 'ns', 'tau', 'mnu', 'wa', 'w', 'bin1', 'bin2', 'bin3', 'bin4', 'bin5', 'bin6']
[0. 1. 3.]
['TT', 'TE', 'EE']
(14, 14)
(8, 8)
15
9


In [18]:
print Fobj.ells
print Fobj.ellsP

[2.000e+00 3.000e+00 4.000e+00 ... 2.998e+03 2.999e+03 3.000e+03]
[   2    3    4 ... 4998 4999 5000]


In [19]:
# define ranges for ell, ellP in Fisher matrices
myLmin = 50; myLmax=3000
myLminP = 50; myLmaxP=5000

In [20]:
# make a noise-free kk-only Fisher matrix
Clkk = Fobj.crossCls[0,0]
dClkk = Fobj.dClVecs[0]
print 'Clkk shape: ',Clkk.shape
print 'dClkk shape: ',dClkk.shape
cov_kk = 2*Clkk**2 / (2*ells+1)
Fij_kk = np.zeros((nCosParams,nCosParams))
minEllInd = myLmin-lmin  # eg. 50-2
for pNum1 in range(nCosParams):
    for pNum2 in range(pNum1,nCosParams):
        # the next line does the summation over ell
        Fij_kk[pNum1,pNum2] = np.sum(dClkk[pNum1,minEllInd:]*dClkk[pNum2,minEllInd:]/cov_kk[minEllInd:])
        Fij_kk[pNum2,pNum1] = Fij_kk[pNum1,pNum2]
print 'parameters: ',Fobj.paramList[:nCosParams]
print 'kappa-only, noise-free Fisher matrix:'
print Fij_kk

Clkk shape:  (2999,)
dClkk shape:  (15, 2999)
parameters:  ['ombh2', 'omch2', 'cosmomc_theta', 'As', 'ns', 'tau', 'mnu', 'wa', 'w']
kappa-only, noise-free Fisher matrix:
[[ 3.36597600e+09 -2.97013769e+09  4.81306485e+10 -7.25322080e+16
  -1.78380957e+08  6.85316634e+04  3.09078162e+07  6.65224434e+07
   7.46243119e+07]
 [-2.97013769e+09  2.62099272e+09 -4.24744577e+10  6.40146781e+16
   1.57379040e+08 -6.04956725e+04 -2.72783892e+07 -5.87062893e+07
  -6.58571766e+07]
 [ 4.81306485e+10 -4.24744577e+10  6.88435671e+11 -1.03795915e+18
  -2.54440217e+09  9.85732071e+05  4.42219892e+08  9.51565863e+08
   1.06753035e+09]
 [-7.25322080e+16  6.40146781e+16 -1.03795915e+18  1.56650847e+24
   3.81307835e+15 -1.50511451e+12 -6.67138841e+14 -1.43485079e+15
  -1.60991427e+15]
 [-1.78380957e+08  1.57379040e+08 -2.54440217e+09  3.81307835e+15
   9.79398669e+06 -3.31945818e+03 -1.62966540e+06 -3.51482806e+06
  -3.94005074e+06]
 [ 6.85316634e+04 -6.04956725e+04  9.85732071e+05 -1.50511451e+12
  -3.3194

# Add noise to kg, k and TE covariance matrices

In [21]:
# get copies of original version with lmin=50
Fij_kg = Fobj.makeFisher(myLmin)
Fij_TE = Fobj.makeFisher(myLminP,TE=True)
print 'done'

adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lm

In [22]:
# copy some code from FisherCl
# there are 3 places in this cell to edit for z0,beesBins,modelDNDZ3

#z0 = 0.3 # for use with cp.modelDNDZ
z0 = 0.5 # for use wiht cp.modelDNDZ3

print 'creating noise arrays... '

print 'getting (EB) lensing reconstruction noise... '
# noise levels from a possible CMB-S4 design:
nlev_t     = 1.   # temperature noise level, in uK.arcmin.
nlev_p     = 1.414   # polarization noise level, in uK.arcmin.
beam_fwhm  = 1.   # Gaussian beam full-width-at-half-maximum.

#ells,EB_noise = ncl.getRecNoise(self.lmax,nlev_t,nlev_p,beam_fwhm)
ellsQC,EB_noise = ncl.getRecNoise(lmax,nlev_t,nlev_p,beam_fwhm)

# convert Nl^dd to Nl^kk
# use Cl^kk = 1/4*[l*(l+1)]^2 * Cl^phiphi?
# but d is in between k and phi, so...?
# Scratch that.
# Assume output of quicklens is Cl^phiphi
#Nlkk = EB_noise * (ells*(ells+1)/2)**2
Nlkk = EB_noise * (ellsQC*(ellsQC+1)/2)**2


print 'getting galaxy shot noise... '
# From Schaan et. al.: LSST n_source = 26/arcmin^2 for full survey
#nbar = 26 # arcmin^-2
nbar = 66 # 66 arcmin^-2 to match Bye's value

# the selection of beesBins must be consistent with that which is selected in cp.tophat
beesBins = True
#beesBins = False
if beesBins:
    binEdges = [0.0,0.5,1.0,2.0,3.0,4.0,7.0]
    nBins = 6
else:
    #binEdges = np.linspace(self.zmin,self.zmax,self.nBins+1)
    #nBins = self.nBins
    binEdges = np.linspace(Fobj.zmin,Fobj.zmax,Fobj.nBins+1)
    nBins = Fobj.nBins
    print 'binEdges: ',binEdges,', nBins: ',nBins

# the selection of dndz function must be consistent with that which is selected in cp.getDNDZinterp
# myDNDZ must be a function only of z
#myDNDZ = lambda z: cp.modelDNDZ(z,z0)
myDNDZ = lambda z: cp.modelDNDZ3(z,z0)

N_gg = ncl.shotNoise(nbar,binEdges,myDNDZ=myDNDZ)



# create noiseCls array to accompany crossCls: 
#  Nl^kk will be reconstruction noise
#  Nl^gigi will be shot noise
#  Nl^kg, Nl^gigj are zero because noise is uncorrelated

#self.noiseCls = np.zeros(self.crossCls.shape)
#self.noiseCls[0,0] = Nlkk[self.lmin:self.lmax+1]
#for binNum in range(nBins):
#    self.noiseCls[binNum+1,binNum+1] = N_gg[binNum]*np.ones(self.lmax-self.lmin+1)
noiseCls = np.zeros(Fobj.crossCls.shape)
noiseCls[0,0] = Nlkk[Fobj.lmin:Fobj.lmax+1]
for binNum in range(nBins):
    noiseCls[binNum+1,binNum+1] = N_gg[binNum]*np.ones(Fobj.lmax-Fobj.lmin+1)


print 'getting (primary CMB) detector noise...'
# CMBS4 v1
fwhm = 1; ST = 1; SP = ST*1.414
#noiseCMBS4_TT1 = ncl.noisePower(ST,ST,fwhm,self.ellsP)
#noiseCMBS4_TP1 = ncl.noisePower(ST,SP,fwhm,self.ellsP)
#noiseCMBS4_PP1 = ncl.noisePower(SP,SP,fwhm,self.ellsP)
noiseCMBS4_TT1 = ncl.noisePower(ST,ST,fwhm,Fobj.ellsP)
#noiseCMBS4_TP1 = ncl.noisePower(ST,SP,fwhm,Fobj.ellsP)
noiseCMBS4_PP1 = ncl.noisePower(SP,SP,fwhm,Fobj.ellsP)

# CMBS4 v2
#fwhm = 2; ST = 1; SP = ST*1.414
#noiseCMBS4_TT2 = ncl.noisePower(ST,ST,fwhm,self.ellsP)
#noiseCMBS4_TP2 = ncl.noisePower(ST,SP,fwhm,self.ellsP)
#noiseCMBS4_PP2 = ncl.noisePower(SP,SP,fwhm,self.ellsP)

# set TE noise to zero
noiseCMBS4_TP1 = np.zeros(Fobj.ellsP.__len__())

# shape like crossCls
noiseCls1 = np.array([[noiseCMBS4_TT1,noiseCMBS4_TP1],[noiseCMBS4_TP1,noiseCMBS4_PP1]])
#noiseCls2 = np.array([[noiseCMBS4_TT2,noiseCMBS4_TP2],[noiseCMBS4_TP2,noiseCMBS4_PP2]])

# pick one
#self.noiseClsP = noiseCls1
noiseClsP = noiseCls1

print noiseCls.shape
print noiseClsP.shape

creating noise arrays... 
getting (EB) lensing reconstruction noise... 
[00:00:00] calculating full-sky noise level for estimator of type <class 'quicklens.qest.lens.phi_EB'>
getting galaxy shot noise... 
getting (primary CMB) detector noise...
(7, 7, 2999)
(2, 2, 4999)


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:64: ComplexWarning: Casting complex values to real discards the imaginary part


In [23]:
print noiseCls[0,0]
print noiseCls[0,1]
print noiseCls[1,1]

print noiseClsP[0,0]
print noiseClsP[0,1]
print noiseClsP[1,1]

[1.44053822e-08 1.15270707e-08 1.06745275e-08 ... 1.28079160e-07
 1.28090978e-07 1.28102819e-07]
[0. 0. 0. ... 0. 0. 0.]
[0.07316074 0.07316074 0.07316074 ... 0.07316074 0.07316074 0.07316074]
[8.46159577e-08 8.46159654e-08 8.46159758e-08 ... 1.23888050e-07
 1.23906952e-07 1.23925861e-07]
[0. 0. 0. ... 0. 0. 0.]
[1.69180807e-07 1.69180823e-07 1.69180843e-07 ... 2.47701271e-07
 2.47739064e-07 2.47776870e-07]


In [24]:
# make covariance matrices with noise
covar, invCov ,ells ,obsList  = Fobj.makeCovar(Fobj.crossCls ,noiseCls ,lmin,myLmax )
covarP,invCovP,ellsP,obsListP = Fobj.makeCovar(Fobj.crossClsP,noiseClsP,lmin,myLmaxP)

print invCov.shape
print invCovP.shape

starting covariance set  1  of  7 ... 
starting covariance set  2  of  7 ... 
starting covariance set  3  of  7 ... 
starting covariance set  4  of  7 ... 
starting covariance set  5  of  7 ... 
starting covariance set  6  of  7 ... 
starting covariance set  7  of  7 ... 
starting covariance set  1  of  2 ... 
starting covariance set  2  of  2 ... 
(28, 28, 2999)
(3, 3, 4999)


# Calculate kg, TE, k Fisher matrices with noise included

In [25]:
# check ell ranges for kappa noise
print ells
print ellsQC

[   2    3    4 ... 2998 2999 3000]
[   0    1    2 ... 2998 2999 3000]


In [26]:
# remake kk covar and fisher with noise

#Clkk = Fobj.crossCls[0,0]
#dClkk = Fobj.dClVecs[0]

# add noise to crossCls for making covar
Clkk2 = Fobj.crossCls[0,0]+Nlkk[lmin:]

print 'Clkk shape: ',Clkk2.shape
print 'dClkk shape: ',dClkk.shape
cov_kk2 = 2*Clkk2**2 / (2*ells+1)
Fij_kk2 = np.zeros((nCosParams,nCosParams))
minEllInd = myLmin-lmin  # eg. 50-2
for pNum1 in range(nCosParams):
    for pNum2 in range(pNum1,nCosParams):
        # the next line does the summation over ell
        Fij_kk2[pNum1,pNum2] = np.sum(dClkk[pNum1,minEllInd:]*dClkk[pNum2,minEllInd:]/cov_kk2[minEllInd:])
        Fij_kk2[pNum2,pNum1] = Fij_kk2[pNum1,pNum2]
print 'parameters: ',Fobj.paramList[:nCosParams]
print 'kappa-only, noise-included Fisher matrix:'
print Fij_kk2

Clkk shape:  (2999,)
dClkk shape:  (15, 2999)
parameters:  ['ombh2', 'omch2', 'cosmomc_theta', 'As', 'ns', 'tau', 'mnu', 'wa', 'w']
kappa-only, noise-included Fisher matrix:
[[ 1.03122666e+08 -9.14472644e+07  1.52908844e+09 -2.50693859e+15
  -2.66796003e+06  4.68712913e+03  1.03431182e+06  2.13308694e+06
   2.41679540e+06]
 [-9.14472644e+07  8.11941234e+07 -1.35945382e+09  2.23426825e+15
   2.31346933e+06 -4.20459452e+03 -9.21349424e+05 -1.89747544e+06
  -2.15076949e+06]
 [ 1.52908844e+09 -1.35945382e+09  2.27963880e+10 -3.75764362e+16
  -3.75842182e+07  7.13138328e+04  1.54865824e+07  3.18381799e+07
   3.61060034e+07]
 [-2.50693859e+15  2.23426825e+15 -3.75764362e+16  6.23577129e+22
   5.74713376e+13 -1.20715874e+11 -2.56749025e+13 -5.25505077e+13
  -5.96560569e+13]
 [-2.66796003e+06  2.31346933e+06 -3.75842182e+07  5.74713376e+13
   1.10661811e+05 -8.30996789e+01 -2.39740649e+04 -5.17470108e+04
  -5.80279032e+04]
 [ 4.68712913e+03 -4.20459452e+03  7.13138328e+04 -1.20715874e+11
  -8.

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:17: ComplexWarning: Casting complex values to real discards the imaginary part


In [27]:
# now the kg, TE ones

# Insert into Fobj to get new Fij, FijTE
Fobj.invCov  = invCov
Fobj.invCovP = invCovP
Fij_kg2 = Fobj.makeFisher(myLmin)
Fij_TE2 = Fobj.makeFisher(myLminP,TE=True)

# restore originals
#Fobj.invCov  = invCov_o
#Fobj.invCovP = invCovP_o
print 'done.'

adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lm

In [28]:
"""
print 'kappa-galaxy Fisher matrix:'
print Fij_kg2
print 'T-E Fisher matrix:'
print Fij_TE2
"""

"\nprint 'kappa-galaxy Fisher matrix:'\nprint Fij_kg2\nprint 'T-E Fisher matrix:'\nprint Fij_TE2\n"

# Save Fobj with noise included

In [29]:
Fobj_X2n = Fobj # is this a copy or an identity?  Don't know; hope it's a copy
Fobj_X2n.covar = covar
Fobj_X2n.covarP = covarP
Fobj_X2n.invCov = invCov
Fobj_X2n.invCovP = invCovP
Fobj_X2n.noiseCls = noiseCls
Fobj_X2n.noiseClsP = noiseClsP
Fobj_X2n.Fij = Fobj_X2n.makeFisher(Fobj.lmin)
Fobj_X2n.FijTE = Fobj_X2n.makeFisher(Fobj.lminP,TE=True)
print 'done.'

adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lm

In [30]:
#"""
import pickle

#filehandler = open('Fisher_7_6X4_w2_wan.obj','w') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
#filehandler = open('Fisher_7_6X4_w2n.obj','w') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
#filehandler = open('Fisher_7_6X4n.obj','w') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
#neutrino_hierarchy = 'normal', usePrimaryCMB = True; from FisherCl branch quickCl
# kludge for this one: told cp.getDNDZinterp to use modelDNDZ3; set cp.tophat beesBins=True
# biasByBin = False (uses sigma(b)/b), dw = 0.005; X2n version has noise included

filehandler = open('Fisher_7_6X2_w2_wan.obj','w') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=True,
#filehandler = open('Fisher_7_6X2_w2n.obj','w') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=True,
#filehandler = open('Fisher_7_6X2n.obj','w') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=True,
#neutrino_hierarchy = 'normal', usePrimaryCMB = True; from FisherCl branch quickCl
# kludge for this one: told cp.getDNDZinterp to use modelDNDZ3; set cp.tophat beesBins=True
# biasByBin = False (uses sigma(b)/b), dw = 0.005; X2n version has noise included

# this one has TE polarazation to l=5000, but only one galaxy bin
#filehandler = open('Fisher_4_1Yn.obj','w') # AccuracyBoost=3, lmax=3000, lmaxP=5000; nz=3e5; nonlinear=True,
#neutrino_hierarchy = 'normal', usePrimaryCMB = True; from FisherCl branch quickCl
# this one used fiducial params and values to match CMB-S4 Science book verification table 8.1

pickle.dump(Fobj_X2n,filehandler)
filehandler.close()
print 'done'
#"""

done


# Create combined Fisher matrices, apply fsky, and convert variables

In [19]:
# create combined Fisher matrices
Fij_kgTE2 = Fij_kg2.copy()
Fij_kgTE2[:nCosParams,:nCosParams] += Fij_TE2

Fij_kTE2 = Fij_kk2.copy()
Fij_kTE2 += Fij_TE2

In [20]:
# apply fsky
fsky = 0.4
Fij_kk2   *= fsky
Fij_kg2   *= fsky
Fij_TE2   *= fsky
Fij_kgTE2 *= fsky
Fij_kTE2  *= fsky

In [21]:
# Convert As to 10^9*As
#"""
AsIndex = 3
mult = 1e-9
def rowColMult(array,index,mult):
    array[index,:] *= mult
    array[:,index] *= mult
    return array
    
Fij_kk2   = rowColMult(Fij_kk2,AsIndex,mult)
Fij_kg2   = rowColMult(Fij_kg2,AsIndex,mult)
Fij_TE2   = rowColMult(Fij_TE2,AsIndex,mult)
Fij_kgTE2 = rowColMult(Fij_kgTE2,AsIndex,mult)
Fij_kTE2  = rowColMult(Fij_kTE2,AsIndex,mult)

# adjust name
paramList[AsIndex] = '10^9 As'
#"""

In [22]:
# transform basis from cosmomc_theta to HO
# use derivative results from "Transform Fisher Matrix with Jacobian"
if nCosParams == 8:
    dthetas = [-2.46099289e-02,  1.05373002e-02,  2.98929388e-05,  # dtheta_dombh2, dtheta_domch2, dtheta_dH0
               0.00000000e+00,  0.00000000e+00,  0.00000000e+00,   # dtheta_dAs, dtheta_dns, dtheta_dtau
               1.94175593e-04,  8.94926461e-04]                    # dtheta_mnu, dtheta_w
elif nCosParams == 9:
    dthetas = [-2.46099289e-02,  1.05373002e-02,  2.98929388e-05,  # dtheta_dombh2, dtheta_domch2, dtheta_dH0
               0.00000000e+00,  0.00000000e+00,  0.00000000e+00,   # dtheta_dAs, dtheta_dns, dtheta_dtau
               1.94175593e-04,  2.43189330e-04,  8.94926461e-04]   # dtheta_mnu, dtheta_wa, dtheta_w
else:
    print 'wtf'

In [23]:
print Fobj.paramList

['ombh2', 'omch2', 'H0', '10^9 As', 'ns', 'tau', 'mnu', 'w', 'bin1']


In [24]:
# The only derivatives in the Jacobian are on the dtheta/dparam row.
mcIndex = 2
jacobian = np.diag(np.ones(nParams))
jacobian[mcIndex,:nCosParams] = dthetas#*100
#jacobian[:,mcIndex] = dthetas#*100

# multiply these all by d{100thetamc}/d{thetamc} = 100
#jacobian[mcIndex,:] *= 100
#jacobian[:,mcIndex] *= 100

print jacobian[:nCosParams,:nCosParams]

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.46099289e-02  1.05373002e-02  2.98929388e-05  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.94175593e-04  8.94926461e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   

In [25]:
# jacobian-it-up
miniJac = jacobian[:nCosParams,:nCosParams]

Fij_kk2j   = np.dot(miniJac.T,np.dot(Fij_kk2,miniJac))
Fij_kg2j   = np.dot(jacobian.T,np.dot(Fij_kg2,jacobian))
Fij_TE2j   = np.dot(miniJac.T,np.dot(Fij_TE2,miniJac))
Fij_kgTE2j = np.dot(jacobian.T,np.dot(Fij_kgTE2,jacobian))
Fij_kTE2j  = np.dot(miniJac.T,np.dot(Fij_kTE2,miniJac))

# adjust name
paramList[mcIndex] = 'H0'

In [26]:
print Fij_kk2.shape

(8, 8)


# Get and compare sigmas

In [27]:
def getSigmas(Fij):
    return np.sqrt(np.diag(np.linalg.inv(Fij)))
    
sigmaskk = getSigmas(Fij_kk2j)
sigmaskg = getSigmas(Fij_kg2j)
sigmasTE = getSigmas(Fij_TE2j)
sigmaskgTE = getSigmas(Fij_kgTE2j)
sigmaskTE = getSigmas(Fij_kTE2j)

In [28]:
from prettytable import PrettyTable

print 'fsky = ',fsky
print 'kappa-only and kappa-galaxies, with noise (biases marg.)'
t = PrettyTable(['Parameter', 'sigmas_kk','sigmas_kg','percent diff.'])
for i,p in enumerate(paramList[:nCosParams]):
    Zi = sigmaskk[i]
    Bi = sigmaskg[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'kappa-galaxies and kappa-galaxies + TE, with noise'
t = PrettyTable(['Parameter', 'sigmas_kg','sigmas_kg+TE','percent diff.'])
for i,p in enumerate(paramList[:nCosParams]):
    Zi = sigmaskg[i]
    Bi = sigmaskgTE[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'TE and kappa + TE, with noise (biases marg.)'
t = PrettyTable(['Parameter', 'sigmas_TE','sigmas_k+TE','percent diff.'])
for i,p in enumerate(paramList[:nCosParams]):
    Zi = sigmasTE[i]
    Bi = sigmaskTE[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'kappa and kappa + TE, with noise (biases marg.)'
t = PrettyTable(['Parameter', 'sigmas_k','sigmas_k+TE','percent diff.'])
for i,p in enumerate(paramList[:nCosParams]):
    Zi = sigmaskk[i]
    Bi = sigmaskTE[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'kappa + TE and kappa-galaxies + TE, with noise (biases marg.)'
t = PrettyTable(['Parameter', 'sigmas_k+TE','sigmas_kg+TE','percent diff.'])
for i,p in enumerate(paramList[:nCosParams]):
    Zi = sigmaskTE[i]
    Bi = sigmaskgTE[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t




fsky =  0.4
kappa-only and kappa-galaxies, with noise (biases marg.)
+-----------+-----------+-----------+---------------+
| Parameter | sigmas_kk | sigmas_kg | percent diff. |
+-----------+-----------+-----------+---------------+
|   ombh2   |   0.354   |   0.272   |     -23.2     |
|   omch2   |    1.18   |   0.899   |     -23.8     |
|     H0    |  9.84e+06 |  1.04e+07 |      5.57     |
|  10^9 As  |    21.7   |    16.5   |     -23.9     |
|     ns    |    2.65   |    2.01   |     -24.2     |
|    tau    |     485   |     400   |     -17.5     |
|    mnu    |    2.77   |    3.25   |      17.4     |
|     w     |     4.4   |    6.02   |      36.8     |
+-----------+-----------+-----------+---------------+
kappa-galaxies and kappa-galaxies + TE, with noise
+-----------+-----------+--------------+---------------+
| Parameter | sigmas_kg | sigmas_kg+TE | percent diff. |
+-----------+-----------+--------------+---------------+
|   ombh2   |   0.272   |   8.45e-05   |      -100     |
|   

# Fix wa to compare to S4 forecasts

In [29]:
#doWa = False
doWa = True
# set to true if loaded file has a wa parameter

Fij_kk2j_nw = Fij_kk2j.copy()
Fij_kg2j_nw = Fij_kg2j.copy()
Fij_TE2j_nw = Fij_TE2j.copy()
Fij_kgTE2j_nw = Fij_kgTE2j.copy()
Fij_kTE2j_nw = Fij_kTE2j.copy()

In [30]:
paramList = Fobj.paramList
print paramList

['ombh2', 'omch2', 'H0', '10^9 As', 'ns', 'tau', 'mnu', 'w', 'bin1']


In [31]:
# define row,col deleting function
def fixParam(Fij,pNum):
    return np.delete(np.delete(Fij,pNum,0),pNum,1)

# ditch wa
if doWa:
    wIndex = 7 # 7 for wa
    Fij_kk2j_nw = fixParam(Fij_kk2j_nw,wIndex)
    Fij_kg2j_nw = fixParam(Fij_kg2j_nw,wIndex)
    Fij_TE2j_nw = fixParam(Fij_TE2j_nw,wIndex)
    Fij_kgTE2j_nw = fixParam(Fij_kgTE2j_nw,wIndex)
    Fij_kTE2j_nw = fixParam(Fij_kTE2j_nw,wIndex)

    paramList = np.delete(paramList,wIndex)
    print paramList

['ombh2' 'omch2' 'H0' '10^9 As' 'ns' 'tau' 'mnu' 'bin1']


In [32]:
# get sigmas
if doWa:
    sigmaskk_nw = getSigmas(Fij_kk2j_nw)
    sigmaskg_nw = getSigmas(Fij_kg2j_nw)
    sigmasTE_nw = getSigmas(Fij_TE2j_nw)
    sigmaskgTE_nw = getSigmas(Fij_kgTE2j_nw)
    sigmaskTE_nw = getSigmas(Fij_kTE2j_nw)


In [33]:
from prettytable import PrettyTable

if doWa:
    print 'fsky = ',fsky
    print 'kappa-only and kappa-galaxies, with noise (biases marg.)'
    t = PrettyTable(['Parameter', 'sigmas_kk','sigmas_kg','percent diff.'])
    for i,p in enumerate(paramList[:nCosParams-1]):
        Zi = sigmaskk_nw[i]
        Bi = sigmaskg_nw[i]
        diff = (Bi-Zi)/Zi *100
        t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
    print t

    print 'kappa-galaxies and kappa-galaxies + TE, with noise'
    t = PrettyTable(['Parameter', 'sigmas_kg','sigmas_kg+TE','percent diff.'])
    for i,p in enumerate(paramList[:nCosParams-1]):
        Zi = sigmaskg_nw[i]
        Bi = sigmaskgTE_nw[i]
        diff = (Bi-Zi)/Zi *100
        t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
    print t

    print 'TE and kappa + TE, with noise (biases marg.)'
    t = PrettyTable(['Parameter', 'sigmas_TE','sigmas_k+TE','percent diff.'])
    for i,p in enumerate(paramList[:nCosParams-1]):
        Zi = sigmasTE_nw[i]
        Bi = sigmaskTE_nw[i]
        diff = (Bi-Zi)/Zi *100
        t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
    print t

    print 'kappa and kappa + TE, with noise (biases marg.)'
    t = PrettyTable(['Parameter', 'sigmas_k','sigmas_k+TE','percent diff.'])
    for i,p in enumerate(paramList[:nCosParams-1]):
        Zi = sigmaskk_nw[i]
        Bi = sigmaskTE_nw[i]
        diff = (Bi-Zi)/Zi *100
        t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
    print t

    print 'kappa + TE and kappa-galaxies + TE, with noise (biases marg.)'
    t = PrettyTable(['Parameter', 'sigmas_k+TE','sigmas_kg+TE','percent diff.'])
    for i,p in enumerate(paramList[:nCosParams-1]):
        Zi = sigmaskTE_nw[i]
        Bi = sigmaskgTE_nw[i]
        diff = (Bi-Zi)/Zi *100
        t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
    print t




fsky =  0.4
kappa-only and kappa-galaxies, with noise (biases marg.)
+-----------+-----------+-----------+---------------+
| Parameter | sigmas_kk | sigmas_kg | percent diff. |
+-----------+-----------+-----------+---------------+
|   ombh2   |   0.354   |   0.272   |     -23.4     |
|   omch2   |    1.18   |   0.899   |     -23.8     |
|     H0    |  7.96e+06 |  6.47e+06 |     -18.7     |
|  10^9 As  |    21.7   |    16.5   |     -24.2     |
|     ns    |    2.64   |       2   |     -24.3     |
|    tau    |     461   |     350   |     -24.1     |
|    mnu    |    2.07   |    2.04   |     -1.14     |
+-----------+-----------+-----------+---------------+
kappa-galaxies and kappa-galaxies + TE, with noise
+-----------+-----------+--------------+---------------+
| Parameter | sigmas_kg | sigmas_kg+TE | percent diff. |
+-----------+-----------+--------------+---------------+
|   ombh2   |   0.272   |   8.14e-05   |      -100     |
|   omch2   |   0.899   |    0.0015    |     -99.8     |
|

# Fix w, bias to compare with Bye's results

In [34]:
#Fij_kk2j_nw = Fij_kk2j.copy()
#Fij_kg2j_nw = Fij_kg2j.copy()
#Fij_TE2j_nw = Fij_TE2j.copy()
#Fij_kgTE2j_nw = Fij_kgTE2j.copy()
#Fij_kTE2j_nw = Fij_kTE2j.copy()

In [35]:
# define row,col deleting function
#def fixParam(Fij,pNum):
#    return np.delete(np.delete(Fij,pNum,0),pNum,1)

# ditch w
wIndex = 7 # 7 for w after removing wa
Fij_kk2j_nw = fixParam(Fij_kk2j_nw,wIndex)
Fij_kg2j_nw = fixParam(Fij_kg2j_nw,wIndex)
Fij_TE2j_nw = fixParam(Fij_TE2j_nw,wIndex)
Fij_kgTE2j_nw = fixParam(Fij_kgTE2j_nw,wIndex)
Fij_kTE2j_nw = fixParam(Fij_kTE2j_nw,wIndex)

paramList = np.delete(paramList,wIndex)
print paramList

IndexError: index 7 is out of bounds for axis 0 with size 7

In [ ]:
# ditch bias
Fij_kk2j_nwb = Fij_kk2j_nw[:7,:7]
Fij_kg2j_nwb = Fij_kg2j_nw[:7,:7]
Fij_TE2j_nwb = Fij_TE2j_nw[:7,:7]
Fij_kgTE2j_nwb = Fij_kgTE2j_nw[:7,:7]
Fij_kTE2j_nwb = Fij_kTE2j_nw[:7,:7]

In [ ]:
# get sigmas
sigmaskk_nw = getSigmas(Fij_kk2j_nw)
sigmaskg_nw = getSigmas(Fij_kg2j_nw)
sigmasTE_nw = getSigmas(Fij_TE2j_nw)
sigmaskgTE_nw = getSigmas(Fij_kgTE2j_nw)
sigmaskTE_nw = getSigmas(Fij_kTE2j_nw)

sigmaskk_nwb = getSigmas(Fij_kk2j_nwb)
sigmaskg_nwb = getSigmas(Fij_kg2j_nwb)
sigmasTE_nwb = getSigmas(Fij_TE2j_nwb)
sigmaskgTE_nwb = getSigmas(Fij_kgTE2j_nwb)
sigmaskTE_nwb = getSigmas(Fij_kTE2j_nwb)


In [ ]:
print 'fsky = ',fsky,', w fixed'
print 'kappa-only and kappa-galaxies, with noise (biases marg.)'
t = PrettyTable(['Parameter', 'sigmas_kk','sigmas_kg','percent diff.'])
if doWa:
    dn = 2
else:
    dn = 1
for i,p in enumerate(paramList[:nCosParams-dn]):
    Zi = sigmaskk_nw[i]
    Bi = sigmaskg_nw[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'kappa-galaxies and kappa-galaxies + TE, with noise'
t = PrettyTable(['Parameter', 'sigmas_kg','sigmas_kg+TE','percent diff.'])
for i,p in enumerate(paramList[:nCosParams-dn]):
    Zi = sigmaskg_nw[i]
    Bi = sigmaskgTE_nw[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'kappa + TE and kappa-galaxies + TE, with noise (biases marg.)'
t = PrettyTable(['Parameter', 'sigmas_k+TE','sigmas_kg+TE','percent diff.'])
for i,p in enumerate(paramList[:nCosParams-dn]):
    Zi = sigmaskTE_nw[i]
    Bi = sigmaskgTE_nw[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t



In [ ]:
#"""
print 'fsky = ',fsky,', w and biases fixed'
print 'kappa-only and kappa-galaxies, with noise'
t = PrettyTable(['Parameter', 'sigmas_kk','sigmas_kg','percent diff.'])
for i,p in enumerate(paramList[:nCosParams-dn]):
    Zi = sigmaskk_nwb[i]
    Bi = sigmaskg_nwb[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'kappa-galaxies and kappa-galaxies + TE, with noise'
t = PrettyTable(['Parameter', 'sigmas_kg','sigmas_kg+TE','percent diff.'])
for i,p in enumerate(paramList[:nCosParams-dn]):
    Zi = sigmaskg_nwb[i]
    Bi = sigmaskgTE_nwb[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'kappa + TE and kappa-galaxies + TE, with noise'
t = PrettyTable(['Parameter', 'sigmas_k+TE','sigmas_kg+TE','percent diff.'])
for i,p in enumerate(paramList[:nCosParams-dn]):
    Zi = sigmaskTE_nwb[i]
    Bi = sigmaskgTE_nwb[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t
#"""


# Export to CosmicFish

In [ ]:
# setup for saving files in directory structure
import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)


In [ ]:
#dirName = 'Z_test_4L' # uses TEk, gk, kgTE with noise; linear P(k)
dirName = 'Z_test_4' # uses TEk, gk, kgTE with noise; nonlinear P(k)
# adjust dirName after creation

createFolder(dirName)

createFolder(dirName+'/parameters_analysis')
createFolder(dirName+'/raw_results')
createFolder(dirName+'/results')

In [ ]:
# fill the raw_results directory
#"""
import os
os.chdir(dirName+'/raw_results')

np.savetxt('1_'+dirName+'_fisher_matrix_Fijkk_cls.dat',Fij_kk2j,fmt='%26.16E', #delimiter='\t',
           header='This file contains a Fisher matrix created with the FisherCl code.')
np.savetxt('2_'+dirName+'_fisher_matrix_Fijkg_cls.dat',Fij_kg2j,fmt='%26.16E', #delimiter='\t',
           header='This file contains a Fisher matrix created with the FisherCl code.')
np.savetxt('3_'+dirName+'_fisher_matrix_FijTE_cls.dat',Fij_TE2j,fmt='%26.16E', #delimiter='\t',
           header='This file contains a Fisher matrix created with the FisherCl code.')
np.savetxt('4_'+dirName+'_fisher_matrix_FijkTE_cls.dat',Fij_kTE2j,fmt='%26.16E', #delimiter='\t',
           header='This file contains a Fisher matrix created with the FisherCl code.')
np.savetxt('5_'+dirName+'_fisher_matrix_FijkgTE_cls.dat',Fij_kgTE2j,fmt='%26.16E', #delimiter='\t',
           header='This file contains a Fisher matrix created with the FisherCl code.')

# add some code to do the .paramnames files too... but not now

os.chdir('../..')
#"""


In [ ]:
# get binBs
binBs = Fobj.binBs
print binBs

In [ ]:
print Fobj.nonlinear

# Load a Planck Fisher Matrix

In [ ]:
# load Planck invCov matrix saved in notebook "Get Planck Parameter Fisher"
invCovPlanck = np.loadtxt('PlanckInvCovFromMCMC.txt')
print invCovPlanck

In [ ]:
# parameters:
# covmat:  [' omegabh2\t\\Omega_b h^2\n', ' omegach2\t\\Omega_c h^2\n', ' theta\t100\\theta_{MC}\n', 
#           ' logA\t{\\rm{ln}}(10^{10} A_s)\n', ' ns\tn_s\n', ' tau\t\\tau\n']
# Note: that third one is 100thetamc, not just thetamc.

# Transform cosmomc_theta parameter matrix to H0 parameter matrix

In [ ]:
# calculation of Jacobian only requires partial derivatives of cosmomc_theta
# This is done in notebook "Transform Fisher Matrix with Jacobian"
# Result is copied here:
dthetaNames =  ['dtheta_dombh2','dtheta_domch2','dtheta_dH0','dtheta_dAs','dtheta_dns','dtheta_dtau']
dthetas = [-0.0248396985185, 0.0104553863834, 2.96219516316e-05, 0.0, 0.0, 0.0]

In [ ]:
# The only derivatives in the Jacobian are on the dtheta/dparam row.
mcIndex = 2
jacobian = np.diag(np.ones(6)) # 6 is nCosParams above
jacobian[mcIndex,:] = dthetas#*100
#jacobian[:,mcIndex] = dthetas#*100

# multiply these all by d{100thetamc}/d{thetamc} = 100
jacobian[mcIndex,:] *= 100
#jacobian[:,mcIndex] *= 100

print jacobian

In [ ]:
# jacobian-it-up
invCovPlanck = np.dot(jacobian.T,np.dot(invCovPlanck,jacobian))
print invCovPlanck

In [ ]:
# ok, also convert ln(10^10*As) to 10^9*As
# exponentiate and divide by 10?  No, use Jacobian again.
# no, actually convert to just As, since that's what FijTE uses
As_fid = 2.2e-9
AsIndex = 3
invCovPlanck[AsIndex,:] *= 1./As_fid
invCovPlanck[:,AsIndex] *= 1./As_fid

# Combine Fisher matrices and show comparison

In [ ]:
FijSum1 = FijTE1t+invCovPlanck
FijSum2 = FijTE2t+invCovPlanck

In [ ]:
# get sigmas
sigmasSum1 = getSigmas(FijSum1)
sigmasSum2 = getSigmas(FijSum2)
# rescale As
sigmasSum1[AsIndex] *= 1e9
sigmasSum2[AsIndex] *= 1e9

In [ ]:
# rescale Planck As for comparison too
sigmasP = getSigmas(invCovPlanck)
sigmasP[AsIndex] *= 1e9

In [ ]:
# alter the As label
paramList[AsIndex] = '10^9 As'

In [ ]:
# alter the ombh2 label and values
paramList[0] = '100ombh2'
sigmasP[0] *= 100
sigmasSum1[0] *= 100
sigmasSum2[0] *= 100

In [ ]:
fiducial = [2.22,0.120,69.0,2.2,0.966,0.06]

In [ ]:
print 'CMB-S4 FWHM = 1 arcmin'
#t = PrettyTable(['Parameter','fiducial','Planck','S4a+Planck','percent diff.'])
t = PrettyTable(['Parameter','fiducial','Planck','S4a+Planck'])
for i,p in enumerate(paramList):
    fid = fiducial[i]
    Zi = sigmasP[i]
    Bi = sigmasSum1[i]
    #diff = (Bi-Zi)/Zi *100
    #t.add_row([p,"%5.3g" % fid,"±%5.3g" % Zi,"±%5.3g" % Bi, "%5.3g" % diff ] )
    t.add_row([p,"%5.3g" % fid,"±%5.3g" % Zi,"±%5.3g" % Bi] )
print t

print 'CMB-S4 FWHM = 2 arcmin'
#t = PrettyTable(['Parameter','fiducial','Planck','S4a+Planck','percent diff.'])
t = PrettyTable(['Parameter','fiducial','Planck','S4b+Planck'])
for i,p in enumerate(paramList):
    fid = fiducial[i]
    Zi = sigmasP[i]
    Bi = sigmasSum2[i]
    diff = (Bi-Zi)/Zi *100
    #t.add_row([p,"%5.3g" % fid,"±%5.3g" % Zi,"±%5.3g" % Bi, "%5.3g" % diff ] )
    t.add_row([p,"%5.3g" % fid,"±%5.3g" % Zi,"±%5.3g" % Bi] )
print t